In [1]:
from shapely.geometry import MultiLineString, LineString, Point, Polygon
from shapely.ops import nearest_points

import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle
import s2g

%load_ext nb_black

INFO:blib2to3.pgen2.driver:Generating grammar tables from D:\Users\sheep\Anaconda3\envs\CFPP_LCA\lib\site-packages\blib2to3\Grammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to C:\Users\sheep\AppData\Local\black\black\Cache\20.8b1\Grammar3.8.8.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: 'C:\\Users\\sheep\\AppData\\Local\\black\\black\\Cache\\20.8b1\\tmphigy07cl'
INFO:blib2to3.pgen2.driver:Generating grammar tables from D:\Users\sheep\Anaconda3\envs\CFPP_LCA\lib\site-packages\blib2to3\PatternGrammar.txt
INFO:blib2to3.pgen2.driver:Writing grammar tables to C:\Users\sheep\AppData\Local\black\black\Cache\20.8b1\PatternGrammar3.8.8.final.0.pickle
INFO:blib2to3.pgen2.driver:Writing failed: [Errno 2] No such file or directory: 'C:\\Users\\sheep\\AppData\\Local\\black\\black\\Cache\\20.8b1\\tmpuat799cs'


<IPython.core.display.Javascript object>

# Railroad data 

In [2]:
file_path = r"D:\tencent_files\chrome_Download\Research\LCA\LCA_CFPP_national\Data"
rail_road_shp = "ne_10m_railroads.shp"

<IPython.core.display.Javascript object>

In [3]:
# extract Asian railroad
rail_world = gpd.read_file(os.path.join(file_path, rail_road_shp))
rail_world.crs = "epsg:4326"
rail_Asia = rail_world[rail_world["continent"] == "Asia"]

# extract China territory
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
CHN_adm = world[world["name"] == "China"]
CHN_adm.to_crs(epsg=4326)

# extract China Railroad
rail_CHN = gpd.sjoin(rail_Asia, CHN_adm, how="inner", op="within")

# transfer from lat-lon measurement to "length" measurement
rail_CHN_m = rail_CHN.to_crs(epsg=32643)  # change measurement into meters
rail_CHN_length = rail_CHN_m.geometry.length  # return a Series Object

<IPython.core.display.Javascript object>

#  Mining site data

In [4]:
mine_data_shp = r"D:\tencent_files\chrome_Download\Research\LCA\LCA Coal_fired power plant phaseout\data\AllChinaCoalMines\AllChinaCoalMines.shp"
mine = gpd.read_file(mine_data_shp).to_crs(epsg=4326)
mine_site = gpd.sjoin(mine, CHN_adm, how="inner", op="within")
len(mine_site)

2412

<IPython.core.display.Javascript object>

# Power Plant site data

In [5]:
power_plant_fp = r"D:\tencent_files\chrome_Download\Research\LCA\LCA_CFPP_national\Data\transport_data"
power_plant_excel_14 = "14_Power_Plant_Data.xlsx"
power_plant_excel_18 = "18_Power_Plant_Data.xlsx"
power_plant_sheet_name = "sheet1"


def read_power_plant_data(
    power_plant_excel,
    power_plant_fp=power_plant_fp,
    power_plant_sheet_name=power_plant_sheet_name,
):

    power_sd = pd.read_excel(
        os.path.join(power_plant_fp, power_plant_excel),
        sheet_name=power_plant_sheet_name,
    )

    power_sd_gdf = gpd.GeoDataFrame(
        power_sd, geometry=gpd.points_from_xy(power_sd.longitude, power_sd.latitude)
    )
    power_sd_gdf.crs = "epsg:4326"
    return power_sd_gdf

<IPython.core.display.Javascript object>

In [6]:
power_sd_gdf_14 = read_power_plant_data(power_plant_excel_14)
power_sd_gdf_18 = read_power_plant_data(power_plant_excel_18)

<IPython.core.display.Javascript object>

# Link point

In [7]:
# link mines to rails
mine_site_m = mine_site.to_crs(epsg=32643)  # transfer to "length" measurement

# transform LineString to MultiLineString
rail_MLS = [rail for rail in rail_CHN_m.geometry]
rail_MLS_temp = MultiLineString(rail_MLS)

# calculate the shortest distance from mines to railway network
shortest_distance_ls = []
intersection_point_ls = []
for site in mine_site_m.geometry:
    shortest_distance = site.distance(rail_MLS_temp)
    intersection_point = nearest_points(site, rail_MLS_temp)[1]
    shortest_distance_ls.append(shortest_distance)
    intersection_point_ls.append(intersection_point)

inter_point_gdf = gpd.GeoDataFrame(
    intersection_point_ls, index=mine_site_m.index, columns=["intersection_point"]
)
shortest_distance_gdf = pd.Series(shortest_distance_ls)
join_mine = mine_site_m.join(inter_point_gdf)
link_mine = join_mine.apply(
    lambda row: LineString([row["geometry"], row["intersection_point"]]), axis=1
)
join_mine["length"] = link_mine.apply(lambda row: row.length)

<IPython.core.display.Javascript object>

In [8]:
len(inter_point_gdf)

2412

<IPython.core.display.Javascript object>

In [19]:
def link_plant_rail(power_sd_gdf):
    # link plant and rail
    power_sd_m = power_sd_gdf.to_crs(epsg=32643)  # 转换坐标系
    intersection_point = nearest_points(power_sd_m.geometry[1], rail_MLS_temp)
    shortest_distance_Power_ls = []
    intersection_point_Power_ls = []
    for site in power_sd_m.geometry:
        shortest_distance = site.distance(rail_MLS_temp)
        intersection_point = nearest_points(site, rail_MLS_temp)[1]
        shortest_distance_Power_ls.append(shortest_distance)
        intersection_point_Power_ls.append(intersection_point)

    inter_point_Power_gdf = gpd.GeoDataFrame(
        intersection_point_Power_ls,
        index=power_sd_m.index,
        columns=["intersection_point"],
    )
    shortest_distance_Power_gdf = pd.Series(shortest_distance_ls)

    join_Power = power_sd_m.join(inter_point_Power_gdf)
    link_Power = join_Power.apply(
        lambda row: LineString([row["geometry"], row["intersection_point"]]), axis=1
    )
    join_Power["length"] = link_Power.apply(lambda row: row.length)
    return join_Power, link_Power, power_sd_m

<IPython.core.display.Javascript object>

In [20]:
join_Power_14, link_Power_14, power_sd_m_14 = link_plant_rail(power_sd_gdf_14)


<IPython.core.display.Javascript object>

In [ ]:
join_Power_18, link_Power_18 = link_plant_rail(power_sd_gdf_18)

In [ ]:
# link all
mine_MLS = [link for link in link_mine]
power14_MLS = [link for link in link_Power_14]
power18_MLS = [link for link in link_Power_18]
link_mine_rail_power14 = gpd.GeoSeries(rail_MLS + mine_MLS + power14_MLS)

In [26]:
link_mine_rail_power14.geometry.to_file(os.path.join(file_path, "link_14.shp"))

<IPython.core.display.Javascript object>

# Transform from geodata to topology graph

In [29]:
shp_file14 = "link_14.shp"
shp_file18 = "link_18.shp"

<IPython.core.display.Javascript object>

In [28]:

links14 = s2g.ShapeGraph(shapefile=os.path.join(file_path,shp_file14), to_graph=True)

INFO:root:Validating pair-wise line connections of raw shapefiles (total 7007 lines)
100% (24545521 of 24545521) |############| Elapsed Time: 2:51:47 Time:  2:51:47


Major components statistics:
	Total components: 12
	Component size: max 6892, median 7.0, min 3, average 583.9166666666666
	Top comp. sizes: 6892 44 18 11 9 8 6 5 4 4


INFO:root:Processing the largest component with 6892 lines
INFO:root:Cutting lines with specific resolution = 1.0 km
100% (6892 of 6892) |####################| Elapsed Time: 0:00:07 Time:  0:00:07
INFO:root:Adding pseudo edges to eliminate gaps between edges
INFO:root:Graph created with 121888 nodes, 121680 edges


<IPython.core.display.Javascript object>

In [31]:
link14_pickle = "link_2014.pickle"
with open(os.path.join(file_path, link14_pickle), "wb") as file:
    pickle.dump(links14, file)

<IPython.core.display.Javascript object>

In [ ]:
links18 = s2g.ShapeGraph(shapefile=os.path.join(file_path,shp_file18), to_graph=True)

In [ ]:
link18_pickle = "link_2018.pickle"
with open(os.path.join(file_path,link18_pickle),'wb') as file:
    pickle.dump(links18,file)

# Calculate Distance

In [13]:
with open(os.path.join(file_path, "link_2014.pickle"), "rb") as file:
    links = pickle.load(file)

<IPython.core.display.Javascript object>

In [17]:
links.node_xy

{0: (3770383.7969564847, 6951620.248816351),
 1: (3770352.509431827, 6951686.790799448),
 2: (3770160.5537673174, 6952317.708803785),
 3: (3770009.2793189846, 6953033.7268798975),
 4: (3769793.4520414528, 6953690.583231862),
 5: (3769696.594475263, 6954313.726562347),
 6: (3769599.8007860086, 6954937.0214893045),
 7: (3769503.0718926, 6955560.563852604),
 8: (3769453.2680758997, 6956131.940086877),
 9: (3769301.2890476556, 6956847.593631209),
 10: (3769323.4919260754, 6957437.366249791),
 11: (3769225.7949448884, 6958060.3621087605),
 12: (3769009.314849856, 6958716.954459946),
 13: (3768809.0040736785, 6959388.18175964),
 14: (3768576.1866307617, 6960029.631666394),
 15: (3768256.2309438987, 6960734.259902514),
 16: (3767808.430858529, 6961416.514619897),
 17: (3767336.3503953787, 6962028.823460932),
 18: (3766918.9936597305, 6962548.839989643),
 19: (3766413.800564973, 6963036.0525052585),
 20: (3765949.192384712, 6963607.713979984),
 21: (3765777.679711302, 6964020.618075023),
 22: 

<IPython.core.display.Javascript object>

In [14]:
link_graph = links.to_networkx()  # transfer graph to networkx object

INFO:root:Processing the largest component with 6892 lines
INFO:root:Cutting lines with specific resolution = 1.0 km
100% (6892 of 6892) |####################| Elapsed Time: 0:00:01 Time:  0:00:01
INFO:root:Adding pseudo edges to eliminate gaps between edges
INFO:root:Graph created with 121888 nodes, 121680 edges


<IPython.core.display.Javascript object>

In [16]:
help(nx.set_node_attributes)

Help on function set_node_attributes in module networkx.classes.function:

set_node_attributes(G, values, name=None)
    Sets node attributes from a given value or dictionary of values.
    
    .. Warning:: The call order of arguments `values` and `name`
        switched between v1.x & v2.x.
    
    Parameters
    ----------
    G : NetworkX Graph
    
    values : scalar value, dict-like
        What the node attribute should be set to.  If `values` is
        not a dictionary, then it is treated as a single attribute value
        that is then applied to every node in `G`.  This means that if
        you provide a mutable object, like a list, updates to that object
        will be reflected in the node attribute for every node.
        The attribute name will be `name`.
    
        If `values` is a dict or a dict of dict, it should be keyed
        by node to either an attribute value or a dict of attribute key/value
        pairs used to update the node's attributes.
    
    nam

<IPython.core.display.Javascript object>

In [18]:
nx.set_node_attributes(link_graph, links.node_xy, "node_xy") #增加node的地理坐标属性

<IPython.core.display.Javascript object>

In [22]:
mine_site_tuple = [tuple(np.array(i)) for i in list(mine_site_m.geometry)]  #
power_tuple_14 = [tuple(np.array(i)) for i in list(power_sd_m_14.geometry)]

<IPython.core.display.Javascript object>

In [23]:
# find nodes of mines and power plants in the graph
selected_nodes_mine = [
    n for n, v in link_graph.nodes(data="node_xy") if v in mine_site_tuple
]
selected_nodes_power_14 = [
    n for n, v in link_graph.nodes(data="node_xy") if v in power_tuple_14
]

<IPython.core.display.Javascript object>

In [25]:
# find coordination of nodes
selected_nodes_mine_coor = pd.Series(
    [Point(link_graph.nodes[i]["node_xy"]) for i in selected_nodes_mine]
)

selected_nodes_power_coor_14 = pd.Series(
    [Point(link_graph.nodes[i]["node_xy"]) for i in selected_nodes_power_14]
)

<IPython.core.display.Javascript object>

In [26]:
# merge nodes and geospatial points of power plants and mines
node_gdf_mine = gpd.GeoDataFrame(
    pd.concat(
        [pd.Series(selected_nodes_mine), selected_nodes_mine_coor], axis=1
    ).rename(columns={0: "node_id", 1: "geometry"})
)
node_gdf_mine.crs = "epsg:32643"

node_gdf_power = gpd.GeoDataFrame(
    pd.concat(
        [pd.Series(selected_nodes_power_14), selected_nodes_power_coor_14], axis=1
    ).rename(columns={0: "node_id", 1: "geometry"})
)
node_gdf_power.crs = "epsg:32643"

<IPython.core.display.Javascript object>

In [27]:
def get_path(graph, source_node, target_node):
    route = nx.shortest_path(G=link_graph, source=source_node, target=target_node)
    coord_ls = []
    for i in range(len(route)):
        coord = Point(link_graph.nodes[route[i]]["node_xy"])
        coord_ls.append(coord)
    path = gpd.GeoSeries(LineString(coord_ls))
    path.crs = "epsg:32643"
    path_length = path.length
    return path, path_length

<IPython.core.display.Javascript object>

In [36]:
os.chdir(file_path)
if not os.path.exists("Distance_matrix"):
    os.mkdir("Distance_matrix")
os.chdir("Distance_matrix")

<IPython.core.display.Javascript object>

In [29]:
def distance_power_to_mine(power_node):
    path_ls = []
    path_length_ls = []
    for i in range(len(selected_nodes_mine)):
        path, path_length = get_path(link_graph, selected_nodes_mine[i], power_node)
        path_ls.append(path)
        path_length_ls.append(path_length)
    path_df = pd.concat(path_ls)
    path_length_df = pd.concat(path_length_ls)
    path_gdf = gpd.GeoDataFrame(pd.concat([path_df, path_length_df], axis=1)).rename(
        columns={0: "geometry", 1: "path_length"}
    )
    with open(str(power_node) + ".pickle", "wb") as data:
        pickle.dump(path_gdf, data)

<IPython.core.display.Javascript object>

In [30]:
%%time
for i in range(0,3):
    distance_power_to_mine(selected_nodes_power_14[i])

Wall time: 10min 18s


<IPython.core.display.Javascript object>

In [ ]:
%%time
for i in range(0,len(selected_nodes_power)):
    distance_power_to_mine(selected_nodes_power[i])